---

## **Problem Statement**

In modern AI-driven applications, it is often necessary to enhance an agent's reasoning by giving it access to real-world information (from the internet). Large Language Models like GPT-4 alone cannot access real-time data. To build a truly intelligent assistant, we need to **augment the model with tools** that bridge these gaps.

---

## **Objectives**

The goal of this application is to:

1. Build a tool-using AI agent that can:
   - Search the web for current news or facts using **Google Search via SerpAPI**
2. Integrate the above tool with **GPT-4 Turbo** using **LangChain’s latest `create_tool_calling_agent` API**
3. Maintain clean, modular, and fully compatible code using the **latest LangChain best practices**
4. Run example queries to demonstrate how the agent utilizes the tool based on the query intent

---

## **Outcomes**

- A functional, intelligent assistant capable of web search.
- Tool usage is decided dynamically by the agent depending on the user’s query.
- Fully integrated with `.env` file for secure API key storage.
- The agent is extensible for further tools (e.g., weather, finance APIs, etc.).

---

In [ ]:
%pip install openai langchain langchain-community python-dotenv -q
# langchain-community – Community-maintained third-party tool integrations for LangChain. Community-maintained integrations like SerpAPI, Wolfram Alpha

In [7]:
%pip install google-search-results -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
%pip install wolframalpha -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv

# LangChain core components
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import Tool

# External API utilities from LangChain Community
from langchain_community.utilities.serpapi import SerpAPIWrapper
# SerpAPIWrapper	Access Google Search via SerpAPI

In [ ]:
# Step 1: Load API keys from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [28]:
# Step 2: Initialize the LLM (GPT-4 Turbo) using OpenAI key
llm = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.3,
    api_key=OPENAI_API_KEY
)

In [29]:
# Step 3: Set up SerpAPI (Google Search wrapper)
search_tool = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)

In [ ]:
# Step 3: Set up SerpAPI (Google Search wrapper)
search_tool = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)

In [ ]:
# Step 4: Wrap SerpAPI as LangChain Tool
tools = [
    Tool.from_function(
        func=search_tool.run,
        name="google_search",
        description="Use this to search the internet for current events, recent information, or news."
    )
    # Only SerpAPI tool is registered
 ]

In [ ]:
# Step 5: Define prompt template (agent_scratchpad is required)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a smart assistant that uses Google Search."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")  # Required for tool-using agents
])

In [40]:
# Step 7: Create the tool-calling agent
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

In [41]:
# Step 8: Wrap agent in an executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [42]:
# Step 9: Run a query through the agent
# def run_agent(query: str):
#     print(f"\nUser: {query}")
#     result = agent_executor.invoke({"input": query})
#     print("Agent:", result["output"])

import asyncio
async def run_agent_async(query: str):
    print(f"\nUser: {query}")
    result = await agent_executor.ainvoke({"input": query})
    print("Agent:", result["output"])

In [43]:
await run_agent_async("What is the square root of 4563?")


User: What is the square root of 4563?


> Entering new AgentExecutor chain...

Invoking: `wolfram_alpha` with `square root of 4563`



Invoking: `wolfram_alpha` with `square root of 4563`


Wolfram Alpha API returned an unexpected response format. Please check your AppID or try again later.Wolfram Alpha API returned an unexpected response format. Please check your AppID or try again later.
Invoking: `wolfram_alpha` with `square root of 4563`
responded: It seems there was an issue retrieving the answer from Wolfram Alpha. Let me try again to get the square root of 4563 for you.


Invoking: `wolfram_alpha` with `square root of 4563`
responded: It seems there was an issue retrieving the answer from Wolfram Alpha. Let me try again to get the square root of 4563 for you.

Wolfram Alpha API returned an unexpected response format. Please check your AppID or try again later.Wolfram Alpha API returned an unexpected response format. Please check your AppID or try again later.I apologize for th

In [12]:
await run_agent_async("Who is the CEO of Twitter as of today?")


User: Who is the CEO of Twitter as of today?


> Entering new AgentExecutor chain...

Invoking: `google_search` with `CEO of Twitter 2023`


['Linda Yaccarino succeeded Musk as CEO on June 5, 2023, with Musk remaining as the chairman and the chief technology officer. In July 2023, Musk announced that ...', 'I am excited to welcome Linda Yaccarino as the new CEO of Twitter! @LindaYacc will focus primarily on business operations, while I focus on ...', 'She took over as CEO of Twitter on June 6, 2023. Fortune and The New Republic described Yaccarino as a former Trump appointee who follows a number of ...', 'Linda Yaccarino, the TV advertising veteran Elon Musk recruited in 2023 to lead business operations at X, is leaving the company, she announced.', 'Linda Yaccarino announced Wednesday that she is stepping down as CEO at X, the social media site formerly known as Twitter.', 'Yaccarino, a longtime advertising executive, took the helm of Twitter in May. Her appointment came after months

In [13]:
await run_agent_async("What is Operation Sindoor?")


User: What is Operation Sindoor?


> Entering new AgentExecutor chain...

Invoking: `google_search` with `Operation Sindoor`


['On 7 May, India launched Operation Sindoor with missile strikes on terrorism-related infrastructure facilities of Pakistan-based militant groups Jaish-e- ...', 'Operation SINDOOR was conceived as a punitive and targeted campaign to dismantle the terror infrastructure across the Line of Control and deeper ...', "The strikes by the Indian Air Force against targets in Pakistan offer a powerful lesson in restraint, and Operation Sindoor adds a new approach to India's ...", 'The Indian military launched multiple missile attacks targeting sites in Pakistan and Pakistan-administered Kashmir early on Wednesday in an attack it called ...', 'Operation Sindoor is a principle-driven military response underpinned by strategic restraint. It was in response to a barbaric terrorist attack ...', "A lightning air campaign that not only crippled Pakistan's air bases but expose

In [14]:
await run_agent_async("what is the per litre petrol price today in Mumbai. Divide the per litre price by 85")


User: what is the per litre petrol price today in Mumbai. Divide the per litre price by 85


> Entering new AgentExecutor chain...

Invoking: `google_search` with `current petrol price per litre in Mumbai`


INR 103.5 per liter
Invoking: `wolfram_alpha` with `103.5 divided by 85`




AssertionError: 

In [ ]:
await run_agent_async("what is the per litre petrol price today in Mumbai. Divide the per litre price by 85. What do you think about the future of petrol cars?")

### Explain the steps to get: SerpAPI Key

Here's a **simple step-by-step guide** to get your **SerpAPI Key** (used for Google Search via API):

---

## What is SerpAPI?

**SerpAPI** is a service that lets you access **Google Search results programmatically** using an API. It's useful for real-time search tasks in AI agents.

---

## Steps to Get Your SerpAPI Key

### **1. Go to SerpAPI's Website**
- Open: [https://serpapi.com](https://serpapi.com)

---

### **2. Sign Up (Free Plan Available)**
- Click on **"Sign Up"** (top right).
- You can sign up using:
  - Google
  - GitHub
  - Email and password

---

### **3. Verify Your Email**
- After signup, check your inbox and click the **verification link** from SerpAPI.

---

### **4. Go to Your Account Dashboard**
- After logging in, you will be redirected to:
  [https://serpapi.com/dashboard](https://serpapi.com/dashboard)

---

### **5. Copy Your API Key**
- On the dashboard homepage, you’ll see:
  - **Your Account Email**
  - **Your Private API Key** (looks like `xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`)
- Click **"Copy"** beside the key.

---

### **6. Paste the Key in `.env` File**
In your project folder, open or create a `.env` file and paste:

```env
SERPAPI_API_KEY=your-copied-key-here
```

Example:
```env
SERPAPI_API_KEY=86e5c04ab2341dcb01de4fc34b33f2e7b640f9d7b00f7f45e
```

---

## Free Tier Info

SerpAPI gives you:
- **100 free searches/month** (for non-commercial use)
- More if you upgrade your plan

---



# Happy Learning